In [1]:
import os
from glob import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from hdbscan import HDBSCAN  # If using HDBSCAN

2025-02-01 14:04:59.733855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Configuration
DATASET_ROOT = "/Volumes/JavaAOT/Documents/AI/ml_expressions/img_datasets/jaffe_dataset"
IMAGE_EXTENSIONS = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']

def get_image_paths(root_dir, extensions):
    """Recursively get all image paths from nested directories"""
    image_paths = []
    for ext in extensions:
        image_paths.extend(glob(os.path.join(root_dir, '**', ext), recursive=True))
    return image_paths

# 1. Initialize Paths (Run This Next)
all_image_paths = get_image_paths(DATASET_ROOT, IMAGE_EXTENSIONS)
print(f"Found {len(all_image_paths)} images in dataset")

Found 213 images in dataset


In [3]:
#1. Emotion-Specific Feature Extraction
# Custom feature extractor focusing on facial expression regions
def create_emotion_feature_extractor():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Focus on mid-level convolutional layers that capture facial expressions
    emotion_features = Model(
        inputs=base_model.input,
        outputs=[
            base_model.get_layer('block4_conv3').output,
            base_model.get_layer('block5_conv3').output
        ]
    )
    return emotion_features

emotion_extractor = create_emotion_feature_extractor()

In [4]:
#2. Facial Region Masking
# Add facial region masking before feature extraction
def apply_emotion_mask(img_array):
    """Focus on eyes, mouth, and eyebrow regions"""
    mask = np.zeros_like(img_array)
    # Define facial regions (coordinates need adjustment for your data)
    mask[75:150, 50:175] = 1  # Eyes and eyebrows
    mask[150:200, 75:150] = 1  # Mouth
    return img_array * mask

# Modify preprocessing
def preprocess_image(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    return apply_emotion_mask(img_array)

In [5]:
#3. Expression-Specific Clustering
def emotion_clustering(features, n_components=20):
    # Combine features
    spatial_features = features[0].mean(axis=(1, 2))
    channel_features = features[1].mean(axis=(1, 2))
    combined_features = np.concatenate([spatial_features, channel_features], axis=1)
    
    # Dimensionality reduction
    pca = PCA(n_components=n_components)
    reduced = pca.fit_transform(combined_features)
    
    # Clustering
    clusterer = HDBSCAN(
        min_cluster_size=50,
        cluster_selection_epsilon=0.5,
        metric='manhattan'
    )
    return clusterer.fit_predict(reduced)

In [6]:
#4. Cluster Validation
def validate_emotion_clusters(clusters, image_paths):
    from collections import defaultdict
    cluster_samples = defaultdict(list)
    
    for idx, cluster in enumerate(clusters):
        if cluster != -1:  # Ignore noise
            cluster_samples[cluster].append(image_paths[idx])
    
    # Display cluster samples with facial landmarks
    for cluster_id, samples in cluster_samples.items():
        print(f"\nCluster {cluster_id} Samples:")
        display_samples(samples[:5])  # Show first 5 samples
        
def display_samples(samples):
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for img_path, ax in zip(samples, axes):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Add facial landmark visualization
        # (Implement actual landmark detection here)
        ax.imshow(img)
        ax.axis('off')
    plt.show()

In [7]:
#5. Execution Pipeline
# Modified workflow
all_image_paths = get_image_paths(DATASET_ROOT, IMAGE_EXTENSIONS)

# 1. Preprocess with emotion masks
batch_images = [preprocess_image(p) for p in all_image_paths]

# 2. Extract emotion-specific features
features = emotion_extractor.predict(np.array(batch_images))

# 3. Emotion-aware clustering
clusters = emotion_clustering(features)

# 4. Validate clusters
validate_emotion_clusters(clusters, all_image_paths)

# 5. Interactive labeling (modified to show emotion landmarks)
@interact(cluster=IntSlider(min=0, max=np.max(clusters), step=1))
def show_emotion_cluster(cluster):
    review_cluster(all_image_paths, clusters, cluster)

7/7 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step


NameError: name 'interact' is not defined